# Collaborative Filtering Recommender System - Expedia Hotel dataset

## Import Libraries

In [239]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
import warnings
import implicit
from scipy.sparse import coo_matrix
import scipy.sparse as sparse

import sys

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
import implicit

In [240]:
df = pd.read_csv('../data/hotel_data/train.csv', sep=',', nrows=1000000)
df.shape

(1000000, 24)

In [241]:
# rename 2 columns
df = df.rename(columns={'hotel_cluster': 'item_id', 'is_booking': 'rating'})

### Remove rows with the same user_id and item_id but different rating

In [243]:
max_rating = df.groupby(['user_id', 'item_id']).rating.transform(max)
df = df.loc[df.rating == max_rating]
df.drop_duplicates(keep='first',inplace=True) 
df= df.reset_index().drop('index',axis=1)

In [244]:
len(df['hotel_market'].unique())

2047

## Assumptions:
    
site_name = 2 -> Expedia.com

posa_continent = 3 -> America

hotel_continent = 2 -> America

hotel_continent = 6-> Europe

user_location_country = 66 -> USA

user_location_country = 205 -> Canada 



hotel_country = 50 -> USA

hotel_country = 8  ->Mexico

hotel_country=198->Canada

hotel_country 70 ->UK

hotel_country 204 ->France

hotel_market 27 Paris

hotel_market= 19-> London.

hotel_market = 675 -> New York 

hotel_market = 701 -> Miami

hotel_market 628 ->Las Vegas

hotel_market 365 ->Los Angeles

hotel_market 1230 ->San Francisco

hotel_market 637 -> Chicago


hotel_market = 212, 214, 213 -> paradise islands

# Create dataframe to store clusters

In [283]:
df[df['hotel_country']==198][['hotel_continent' ,'hotel_country' ,'hotel_market' ,'item_id' ]]

,hotel_continent,hotel_country,hotel_market,item_id
259,2,198,391,68
260,2,198,391,40
261,2,198,391,83
262,2,198,391,37
263,2,198,391,72
...,...,...,...,...
762284,2,198,399,10
762285,2,198,399,42
762324,2,198,399,9
762325,2,198,399,49


In [245]:
hotel_df = pd.DataFrame(columns=['item_id','hotel_name'])
hotel_df['item_id']=list(range(100))
cluster=[]

### Check information of hotel_market 
Add until now: hotel_market = 212,213,214 , item_id = (0,26,34,73)

In [263]:
df[df['hotel_market']==214][['hotel_continent' ,'hotel_country' ,'hotel_market' ,'item_id' ]]

,hotel_continent,hotel_country,hotel_market,item_id
58,2,50,214,28
59,2,50,214,73
60,2,50,214,26
61,2,50,214,28
5585,2,50,214,26
...,...,...,...,...
761343,2,50,214,84
761344,2,50,214,0
761345,2,50,214,26
761346,2,50,214,26


## The most repeated number of item_id (hotel_cluster) with this hotel_market number

In [264]:
df[df['hotel_market']==214]['item_id'].value_counts()[:10]

26    199
73    190
34    161
0     157
84    140
28     67
10     54
59     49
32     47
92     46
Name: item_id, dtype: int64

# Find Similar Hotel clusters

In [265]:
#csr_matrix((data, (row, col))
sparse_item_user = sparse.csr_matrix((df['rating'].astype(float),(df['item_id'], df['user_id'])))
sparse_user_item = sparse.csr_matrix((df['rating'].astype(float),(df['user_id'], df['item_id'])))


model = implicit.als.AlternatingLeastSquares(factors=20,regularization=0.1,iterations=20)
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')
model.fit(data_conf)

item_id = 34
n_similar = 6
similar = model.similar_items(item_id,n_similar)
similar
#for item in similar:
#    idx,score = item
#    print(df.item_id.loc[df.item_id == idx].iloc[0])

[(34, 0.46054688),
 (96, 0.3630649),
 (26, 0.2983222),
 (84, 0.2849912),
 (80, 0.2535136),
 (73, 0.25050712)]

In [266]:
for i in similar:
    cluster.append(i[0])

In [285]:
set(cluster)

{0, 17, 26, 31, 34, 66, 73, 80, 84, 87, 92, 96}

In [272]:
cluster_item_id_0_26_34_73 =[0,96,34,31,84,87,26,96,80,84,92,66,73,80,92,17,84,26,34,96,26,84,80,73]

## Check this hotel cluster

In [279]:
for i in cluster_item_id_0_26_34_73:
    hotel_df['hotel_name'][i]="beach_resorts"


In [280]:
hotel_df

,item_id,hotel_name
0,0,beach_resorts
1,1,NaN
2,2,NaN
3,3,NaN
4,4,NaN
...,...,...
95,95,NaN
96,96,beach_resorts
97,97,NaN
98,98,NaN
